In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def generate_drop_shadow(image, shadow_offset=(10, 10), shadow_opacity=0.7):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the grayscale image to create a binary mask
    _, thresholded_mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

    # Find the contours of the object
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask to draw the shadows on
    shadow_mask = np.zeros_like(image)

    # Draw shadows for each contour
    for contour in contours:
        # Create a copy of the image
        shadow_image = image.copy()

        # Find the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Create a shadow by darkening the region outside the contour
        shadow_image[thresholded_mask == 0] = (shadow_image[thresholded_mask == 0] * shadow_opacity).astype(np.uint8)

        # Offset the shadow
        M = np.float32([[1, 0, shadow_offset[0]], [0, 1, shadow_offset[1]]])
        shadow_image = cv2.warpAffine(shadow_image, M, (shadow_image.shape[1], shadow_image.shape[0]))

        # Combine the shadow with the original image
        shadow_mask = cv2.addWeighted(shadow_mask, 1, shadow_image, 1, 0)

    # Combine the original image with the drop shadow mask
    result = cv2.addWeighted(image, 1, shadow_mask, 0.8, 0)

    return result

# Load the isolated object (car) image
image_path = 'path/to/your/car_image.png'
image = cv2.imread(image_path)

# Generate the drop shadow for the car
result_image = generate_drop_shadow(image)

# Display both original and shadowed images side by side
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Display the original image
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image')
axes[0].axis('off')

# Display the shadowed image
axes[1].imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
axes[1].set_title('Shadowed Image')
axes[1].axis('off')

plt.show()
